In [1]:
import numpy as np
import cv2 as cv
import random
import pickle
import glob
import os
import shutil
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,AveragePooling2D, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

%matplotlib inline

In [2]:
base_dir = r"C:\Users\Reon\Documents\DLSL Files\Thesis Training\Dataset3aug"
testdata_dir = r"C:\Users\Reon\Documents\DLSL Files\Thesis Training\Datasettest"
base_dir2 = r"C:\Users\Reon\Documents\DLSL Files\Thesis Training\Datasetnonaug\trainval"
testdata_dir2 = r"C:\Users\Reon\Documents\DLSL Files\Thesis Training\Datasetnonaug\trainvaltest"
train_dir = base_dir

# Directory with our training/validating pictures --original + augmented--
train_AD_dir = os.path.join(base_dir, 'Atopic Dermatitis')
train_Pso_dir = os.path.join(base_dir, 'Psoriasis')
train_ND_dir = os.path.join(base_dir, 'Nummular Dermatitis')
train_CD_dir = os.path.join(base_dir, 'Contact Dermatitis')
train_RW_dir = os.path.join(base_dir, 'Ringworm')
train_NG_dir = os.path.join(base_dir, 'Negatives')
# Directory with our test pictures --original + augmented--
test_AD_dir = os.path.join(testdata_dir, 'Atopic Dermatitis')
test_Pso_dir = os.path.join(testdata_dir, 'Psoriasis')
test_ND_dir = os.path.join(testdata_dir, 'Nummular Dermatitis')
test_CD_dir = os.path.join(testdata_dir, 'Contact Dermatitis')
test_RW_dir = os.path.join(testdata_dir, 'Ringworm')
test_NG_dir = os.path.join(testdata_dir, 'Negatives')

# Directory with our training/validating pictures --original--
train2_AD_dir = os.path.join(base_dir2, 'Atopic Dermatitis')
train2_Pso_dir = os.path.join(base_dir2, 'Psoriasis')
train2_ND_dir = os.path.join(base_dir2, 'Nummular Dermatitis')
train2_CD_dir = os.path.join(base_dir2, 'Contact Dermatitis')
train2_RW_dir = os.path.join(base_dir2, 'Ringworm')
train2_NG_dir = os.path.join(base_dir2, 'Negatives')
# Directory with our test pictures --original--
test2_AD_dir = os.path.join(testdata_dir2, 'Atopic Dermatitis')
test2_Pso_dir = os.path.join(testdata_dir2, 'Psoriasis')
test2_ND_dir = os.path.join(testdata_dir2, 'Nummular Dermatitis')
test2_CD_dir = os.path.join(testdata_dir2, 'Contact Dermatitis')
test2_RW_dir = os.path.join(testdata_dir2, 'Ringworm')
test2_NG_dir = os.path.join(testdata_dir2, 'Negatives')

train_AD_fnames = os.listdir(train_AD_dir)
print(train_AD_fnames[:10])

train_Pso_fnames = os.listdir(train_Pso_dir)
train_Pso_fnames.sort()
print(train_Pso_fnames[:10])

train_ND_fnames = os.listdir(train_ND_dir)
print(train_ND_fnames[:10])

train_RW_fnames = os.listdir(train_RW_dir)
train_RW_fnames.sort()
print(train_RW_fnames[:10])

train_CD_fnames = os.listdir(train_CD_dir)
print(train_CD_fnames[:10])

train_NG_fnames = os.listdir(train_NG_dir)
train_NG_fnames.sort()
print(train_NG_fnames[:10])

print('# of images in train Original + Augmented Dataset: ')
print('total Atopic Dermatitis images:', len(os.listdir(train_AD_dir)))
print('total Psoriasis images:', len(os.listdir(train_Pso_dir)))
print('total Nummular Dermatitis images:', len(os.listdir(train_ND_dir)))
print('total Contact Dermatitis images:', len(os.listdir(train_CD_dir)))
print('total Ringworm:', len(os.listdir(train_RW_dir)))
print('total Negatives:', len(os.listdir(train_NG_dir)))
print('# of images in test Original + Augmented Dataset: ')
print('total Atopic Dermatitis images:', len(os.listdir(test_AD_dir)))
print('total Psoriasis images:', len(os.listdir(test_Pso_dir)))
print('total Nummular Dermatitis images:', len(os.listdir(test_ND_dir)))
print('total Contact Dermatitis images:', len(os.listdir(test_CD_dir)))
print('total Ringworm:', len(os.listdir(test_RW_dir)))
print('total Negatives:', len(os.listdir(test_NG_dir)))
trainval_total = len(os.listdir(train_AD_dir)) + len(os.listdir(train_Pso_dir)) + len(os.listdir(train_CD_dir)) + len(os.listdir(train_RW_dir)) + len(os.listdir(train_ND_dir)) + len(os.listdir(train_NG_dir))
testval_total = len(os.listdir(test_AD_dir)) + len(os.listdir(test_Pso_dir)) + len(os.listdir(test_CD_dir)) + len(os.listdir(test_RW_dir)) + len(os.listdir(test_ND_dir)) + len(os.listdir(test_NG_dir))
print('Total Images for train original + aug train/val:', trainval_total)
print('Total Images for test original + aug train/val:', testval_total)
print()
print()

print('# of images in train Original Dataset: ')
print('total Atopic Dermatitis images:', len(os.listdir(train2_AD_dir)))
print('total Psoriasis images:', len(os.listdir(train2_Pso_dir)))
print('total Nummular Dermatitis images:', len(os.listdir(train2_ND_dir)))
print('total Contact Dermatitis images:', len(os.listdir(train2_CD_dir)))
print('total Ringworm:', len(os.listdir(train2_RW_dir)))
print('total Negatives:', len(os.listdir(train2_NG_dir)))
print('# of images in test Original Dataset: ')
print('total Atopic Dermatitis images:', len(os.listdir(test2_AD_dir)))
print('total Psoriasis images:', len(os.listdir(test2_Pso_dir)))
print('total Nummular Dermatitis images:', len(os.listdir(test2_ND_dir)))
print('total Contact Dermatitis images:', len(os.listdir(test2_CD_dir)))
print('total Ringworm:', len(os.listdir(test2_RW_dir)))
print('total Negatives:', len(os.listdir(test2_NG_dir)))
trainval2_total = len(os.listdir(train2_AD_dir)) + len(os.listdir(train2_Pso_dir)) + len(os.listdir(train2_CD_dir)) + len(os.listdir(train2_RW_dir)) + len(os.listdir(train2_ND_dir)) + len(os.listdir(train2_NG_dir))
testval2_total = len(os.listdir(test2_AD_dir)) + len(os.listdir(test2_Pso_dir)) + len(os.listdir(test2_CD_dir)) + len(os.listdir(test2_RW_dir)) + len(os.listdir(test2_ND_dir)) + len(os.listdir(test2_NG_dir))
print('Total Images for train original + aug train/val:', trainval2_total)
print('Total Images for test original + aug train/val:', testval2_total)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Reon\\Documents\\DLSL Files\\Thesis Training\\Dataset3aug\\Atopic Dermatitis'

In [3]:
# Model Values and Parameters
img_width = 224
img_height = 224
channels = 3

input_size = (img_width, img_height, channels)
num_classes = 6
epochs = 50
batch_size = 16
validation_split_size = 0.2;
useAllSamples = True ## True to Consume All Traindata per Epoch (for feature crucial classification like defects)
learningRate = 0.000001; ## VGG: 0.000001
fixedStepSize = 2 ## If useAllSamples is False, get no. steps per epoch
patience = 10 
layersToUnfreeze = 12

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
    
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                                   #horizontal_flip=True,
                                   #vertical_flip=True,
                                   #featurewise_center=True,
                                   #featurewise_std_normalization=True,
                                   #validation_split=validation_split_size)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                                  #validation_split=validation_split_size)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        base_dir2,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='categorical',
        subset = 'training',
        shuffle=True)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        base_dir2,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=True)

#Model 1
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import preprocess_input
sourceModel = keras.applications.VGG16

#Early stopping callback in training
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=patience)

vgg16=sourceModel(weights='imagenet', 
                              include_top=False, 
                              input_shape = input_size); 
skinmodel=vgg16.output
addedLayers = 0;

skinmodel = GlobalAveragePooling2D()(skinmodel); addedLayers = addedLayers + 1;
skinmodel=Dense(4096,activation='relu')(skinmodel)
addedLayers = addedLayers + 1
skinmodel=Dropout(0.65)(skinmodel)
addedLayers = addedLayers + 1
skinmodel=Dense(4096,activation='relu')(skinmodel)
addedLayers = addedLayers + 1
skinmodel=Dense(4096,activation='relu')(skinmodel)
addedLayers = addedLayers + 1        
    
preds=Dense(num_classes,activation='softmax')(skinmodel); addedLayers = addedLayers + 1;
model=Model(inputs=vgg16.input,outputs=preds)
for layer in model.layers[0:-(addedLayers+layersToUnfreeze)]:
    layer.trainable = False
model.summary()
#Show Architecture
for i,layer in enumerate(model.layers):
    print(i,layer.name,layer.trainable)

Found 2400 images belonging to 6 classes.
Found 0 images belonging to 6 classes.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________

In [ ]:
#Compiling Model
opt = tf.keras.optimizers.SGD(learning_rate=learningRate, momentum = 0.8)
#opt = tf.keras.optimizers.RMSprop(learning_rate=learningRate, momentum = 0.8, centered = True)      
#opt = tf.keras.optimizers.Adam(learning_rate=learningRate)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

# Fitting Model

#if useAllSamples:
#    step_size_train=train_generator.n//train_generator.batch_size
#else:
#    step_size_train=fixedStepSize


results = model.fit(
    train_generator,
#    validation_data = validation_generator,
    epochs = epochs,
#    steps_per_epoch=step_size_train,
    batch_size = batch_size,
    callbacks = [callback]
)

In [ ]:
# Training and Test Accuracy
print("Training Accuracy of the model is - " , model.evaluate(train_generator)[1]*100 , "%")
print("Validation Accuracy of the model is - " , model.evaluate(validation_generator)[1]*100 , "%")

# Loss and Accuracy Plots

fig, ax = plt.subplots(1, 2, sharex='col', figsize = (15,5))

# loss
ax[0].set_title('Loss', size = 14)
ax[0].plot(results.history['loss'], label='train loss')
ax[0].plot(results.history['val_loss'], label='val loss')
ax[0].legend()

# accuracies
ax[1].set_title('Accuracy', size = 14)

ax[1].plot(results.history['acc'], label='train acc')
ax[1].plot(results.history['val_acc'], label='val acc')
ax[1].legend()

plt.show()

### Save Model
model.save('Skin Disease Classifier.h5', 
        overwrite=True,
        include_optimizer=True
)
model.save_weights("initial_weights.h5")

#### Save Label Names
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels);

fid = open('label_vgg16.sav','wb');
pickle.dump(labels, fid);
fid.close();


print("Model successfully trained and saved.");


In [ ]:
import numpy as np
import cv2 as cv
import random
import pickle
import glob
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

dataPath = 'Datasetnonaug/trainval/'
ext = [".jpg", ".jpeg", ".png"]

#from tensorflow.keras.preprocessing import image
imgsz = 224; ## 224 resnet and vgg, 299 for inception
from tensorflow.keras.applications.vgg16 import preprocess_input
sourceModel = keras.applications.VGG16
    
    
modelName = "Skin Disease Classifier.h5";


testmodel = tf.keras.models.load_model(
        modelName)
    #    custom_objects=None,
    #    compile=True
    #)

f = open("label_vgg16.sav",'rb');labels = pickle.load(f);f.close();
labels = list(labels.values());

X = [];
y = [];


for label in labels:
        fileList = []
        for ctrext in range(0,len(ext)):
            
            fileList.extend(glob.glob(dataPath[:-1] + "test" + "/" + label + "/*" + ext[ctrext]))
        
        
        
        for thisFile in fileList:
            ## CV IMAGE LOAD
            img = cv.imread(thisFile);
            img = cv.cvtColor(img,cv.COLOR_BGR2RGB);
            
          
            img = cv.resize(img,(imgsz,imgsz));
            
            
            img = preprocess_input(img);
            
            X.append(img);
            y.append(labels.index(label));
            
        

y_preds = testmodel.predict(np.array(X));
y_pred = [];
for y_preds_i in y_preds:
        y_pred.append(np.argmax(y_preds_i));
#y_pred = testmodel.predict(X);

accuracy = sum(np.equal(y_pred,y)) / len(y) * 100

print()
print("RANK 1 Accuracy Against Test Data: " + "{:.2f}".format(accuracy) + " %");


confmat = confusion_matrix(y, y_pred)

print()
print('CONFUSION MATRIX:')
disp = ConfusionMatrixDisplay(confusion_matrix=confmat,
                              display_labels=labels)
disp.plot(xticks_rotation='vertical',values_format='')
plt.show()

print();
print("CLASSIFICATION REPORT:")
print(classification_report(y, y_pred))




## SHOW OUTPUT BREAKDOWN
print()
print("OUTPUT DISTRIBUTION:")
outctr = 0;
total = sum(sum(confmat));
for col in confmat.T:
        
        totalcol = sum(col)
        
        print(labels[outctr] + ": " + ("%1.2f") % ((totalcol / total) * 100) + "%")
        
        outctr = outctr + 1;